# Obligatorio 2 - Big Data Science

Integrantes del grupo: Federico Abdo, Juan Cruz Carrau, Joaquín Fernández

Debajo de cada pregunta o tarea incluya las celdas necesarias para desarrolar la respuesta. Puede usar una o varias celdas de código o mark down (https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook)

Para entregar, renombrar este notebook como "Obligatorio 2 - Apellido1 - Apellido 2 - Apellido 3" con los apellidos de los miembros del grupo. Un solo integrante del grupo debe realizar la entrega. 


1. Cargar los datos del Obligatorio 1, de entrenamiento (.data) y validación (.test) en spark dataframes (distintos). Los nombres de las columnas deben corresponder a los especificados en "Attribute Information".

In [10]:
from pyspark.context import SparkContext
sc.stop()
sc = SparkContext('local', 'test')

In [11]:
sc

<SparkContext master=local appName=test>

In [12]:
from pyspark.sql import SQLContext

spark = SQLContext(sc)

In [13]:
spark

In [14]:
from pyspark.sql.types import *

# Schema de los dataset
schema = StructType([
    StructField(name='age', dataType=IntegerType(), nullable=False),
    StructField(name='workclass', dataType=StringType(), nullable=False),
    StructField(name='fnlwgt', dataType=StringType(), nullable=False), # Se pone en tipo string porque al ponerlo en tipo Integer o Long carga la columna con valores null
    StructField(name='education', dataType=StringType(), nullable=False),
    StructField(name='education_num', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='marital-status', dataType=StringType(), nullable=False),
    StructField(name='occupation', dataType=StringType(), nullable=False),
    StructField(name='relationship', dataType=StringType(), nullable=False),
    StructField(name='race', dataType=StringType(), nullable=False),
    StructField(name='sex', dataType=StringType(), nullable=False),
    StructField(name='capital_gain', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='capital_loss', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='hours_per_week', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='native_country', dataType=StringType(), nullable=False),
    StructField(name='income', dataType=StringType(), nullable=False)
])

In [15]:
df_adult = spark.read.csv("adult.data", header=True, schema=schema)
df_test = spark.read.csv("adult.test", header=True, schema=schema) # sacamos el punto del archivo adult.test

In [16]:
df_adult.show(10)

+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass| fnlwgt|    education|education_num|      marital-status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 50| Self-emp-not-inc|  83311|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|           0|           0|            13| United-States| <=50K|
| 38|          Private| 215646|      HS-grad|            9|            Divorced| Handlers-cleaners| Not-in-family| White|   Male|           0|           0|            40| United-States| <=50K|
| 53|          Private| 234721|    

In [17]:
df_test.show(10)

+---+-----------------+-------+-------------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass| fnlwgt|    education|education_num|     marital-status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+-----------------+-------+-------------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 38|          Private|  89814|      HS-grad|            9| Married-civ-spouse|   Farming-fishing|       Husband| White|   Male|           0|           0|            50| United-States| <=50K|
| 28|        Local-gov| 336951|   Assoc-acdm|           12| Married-civ-spouse|   Protective-serv|       Husband| White|   Male|           0|           0|            40| United-States|  >50K|
| 44|          Private| 160323| Some-col

2. Seleccionar un conjunto relevante de 5 atributos y crear un Spark Pipeline en el que el estimator sea un DecisionTreeClassifier (https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier). Puede utilizar libremente los transformers/estimators de Spark para realizar ingeniería de atributos (StringIndexer, OneHotEncoding, etc).

In [18]:
atributos = ["age", "sex", "occupation", "education", "workclass"]

#Primero hay que bucketizar la edad. La separamos en 3 buckets. 

def calculo_de_buckets(df, cantidad_buckets, columna):
    df.createOrReplaceTempView("df_view")
    instancias_totales = spark.sql('''SELECT count(*) as count FROM df_view''').first()['count']
    instancias_en_bucket = instancias_totales / cantidad_buckets
    
    limites = []
    ultimo_limite_superior = None
    for limit in range(0, instancias_totales - 1, round(instancias_en_bucket)): # se pone un -1 porque instancias totales es el count pero queremos llegar hasta el indice        
        query = spark.sql(f'''SELECT {columna} FROM df_view ORDER BY 1 asc''').collect()
        limite_inferior = query[limit][columna]
        
        if(ultimo_limite_superior is not None):
            if ultimo_limite_superior == limite_inferior:
                limite_inferior += 1;
        
        limite = [limite_inferior, query[limit + round(instancias_en_bucket)][columna]]
        limites.append(limite)
        
        ultimo_limite_superior = query[limit + round(instancias_en_bucket)][columna]

    return limites

limites = calculo_de_buckets(df_adult, 3, "age")

In [19]:
from pyspark.sql import functions as F

def age_bucket_udf(age):
    indexes = range(len(limites))
    for index, limite in zip(indexes, limites):
        if age in range(limite[0], limite[1] + 1):    #Se agrega el +1 para hacer el range inclusivo superiormente
            return str(index)

age_udf = F.udf(age_bucket_udf, StringType())

df_adult_bucketized = df_adult.withColumn("ageBucket", age_udf(F.col("age")))
df_test_bucketized = df_test.withColumn("ageBucket", age_udf(F.col("age")))

In [20]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler

In [21]:
def get_indexers(columnas):
    
    list_indexer = []
    
    for columna in columnas:
        list_indexer.append(StringIndexer(inputCol=columna, outputCol=columna+"Index"))
    
    return list_indexer

In [22]:
def get_pipeline(
                categoricalColumns=["sex", "occupation", "education", "workclass", "ageBucket"],
                labelCol='income',
                thresholds=None
                ):
    stages = []

    stages += get_indexers([labelCol])

    stages += get_indexers(categoricalColumns)

    assembler_input = [c + "Index" for c in categoricalColumns]

    assembler = VectorAssembler(
        inputCols=assembler_input,
        outputCol="features")

    stages += [assembler]
    
    dt = DecisionTreeClassifier(labelCol=labelCol+"Index", featuresCol="features")
    
    if thresholds:
        dt.setThresholds(thresholds)

    stages += [dt]
    
    return Pipeline(stages=stages)

In [23]:
pipeline = get_pipeline()

3. Hacer el fit del pipeline con los datos de entrenamiento. 


In [15]:
trainingData = df_adult_bucketized
testData = df_test_bucketized

In [16]:
model = pipeline.fit(trainingData)

KeyboardInterrupt: 

In [ ]:
# Analisis extra

In [ ]:
predictions = model.transform(testData)

In [ ]:
predictions.show(10, truncate=False)

In [ ]:
df_ = predictions

df_.createOrReplaceTempView('df_')

df_aciertos = spark.sql("""
    SELECT income, prediction, count(*)
    FROM df_
    GROUP BY 1, 2
""")

df_aciertos.show()

In [ ]:
# MATRIZ DE CONFUSION
import pandas as pd
pd.DataFrame([[1109, 535], [2737, 11899]], columns=['POSITIVO ACTUAL', 'NEGATIVO ACTUAL'], index = ['POSITIVO PREDECIDO', 'NEGATIVO PREDECIDO'])

4. Graficar la curva ROC utilizando los datos de validación (sin usar el paquete de evluación de Spark pyspark.ml.evaluation)

In [ ]:
trainingData = df_adult_bucketized
testData = df_test_bucketized

In [ ]:
def get_tpr_and_fpr(prediction):
    
    df_ = predictions

    df_.createOrReplaceTempView('df_')

    df_aciertos = spark.sql("""
        SELECT income, prediction, count(*) as count
        FROM df_
        GROUP BY 1, 2
    """)
    
    tp = df_aciertos.where('income = " >50K" and prediction = 1.0').first()['count'] if df_aciertos.where('income = " >50K" and prediction = 1.0').count() > 0 else 0
    
    fn = df_aciertos.where('income = " >50K" and prediction = "0.0"').first()['count'] if df_aciertos.where('income = " >50K" and prediction = "0.0"').count() > 0 else 0
    
    
    fp = df_aciertos.where('income = " <=50K" and prediction = 1.0').first()['count'] if df_aciertos.where('income = " <=50K" and prediction = 1.0').count() > 0 else 0
    
    
    tn = df_aciertos.where('income = " <=50K" and prediction = 0.0').first()['count'] if df_aciertos.where('income = " <=50K" and prediction = 0.0').count() > 0 else 0
    
        
    tpr = tp/(tp + fn)
    
    fpr = fp/(tn + fp)
    
    return (tpr, fpr)

In [ ]:
get_tpr_and_fpr(predictions)

In [ ]:
thresholds = range(0, 105, 5)

values = []

for t in thresholds:
    t = t/100
    print(t)
    
    pipeline = get_pipeline(thresholds=[1-t, t])
    
    model = pipeline.fit(trainingData)
    
    predictions = model.transform(testData)
    
    tpr, fpr = get_tpr_and_fpr(predictions)
    
    values.append([t, [tpr, fpr]])    

In [ ]:
values

In [ ]:
df = pd.DataFrame([(p,tpr,fpr) for (p,(tpr,fpr)) in values], columns=['p', 'tpr', 'fpr'])

In [ ]:
df.plot(style='.-',x='fpr',y='tpr', 
        title="ROC (Receiver Operating Characteristic) Curve")

5. Escribir dos funciones train() y predict() que creen el arbol de acuerdo a la metodología vista en clase (utilizando entropía como métrica de homogeneidad de clases).
```
def train(train_dataframe):
    '''
    @return devuelve una estructura de datos que representa el árbol de decision
    '''
     pass        
```

```
def predict(tree, train_dataframe)
    '''
    @param tree la estructura de datos que representa el árbol de decisión.
    @ return un dataframe con todos los datos de train_dataframe con una columna adicional que representa la probabilidad de que el income sea >50K. 
    '''
    pass
```    
Ejemplo de uso:

```
tree = train(train_dataframe)
predictions_df = predict(tree, train_dataframe)
```

In [24]:
import math
def entropy(probabs):
    return -sum([ p * math.log(p,2) for p in probabs if p != 0])

In [25]:
def get_feature_entropy(df, col):
    """Calculamos la entropy de una columna"""
    
    ncol = df.select(col).distinct().collect()
    ncol = [row[col] for  row in ncol]
    
    total = df.count()
    
    prob_valor = {}
    
    entropy_valor = {}
    
    for valor in ncol:
        ntmp = df.select(col).where(f'{col} = "{valor}"').count()
        
        prob_valor[valor] = ntmp/total
        
        ntmp_pos = df.select(col).where(f'{col} = "{valor}" and income = " >50K"').count()
        
        prob_pos = ntmp_pos/ntmp
        
        prob_neg = 1 - prob_pos
        
        entropy_ = entropy([prob_pos, prob_neg])
        
        entropy_valor[valor] = entropy_
    
    suma_ponderada = 0
    
    for valor in ncol:
        suma_ponderada += prob_valor[valor] * entropy_valor[valor]
    
    return suma_ponderada

In [215]:
def make_split(df, root, categorias):
    
    print(vars(root))
    
    if len(categorias) == 0:
        
        c1 = df.select('income').where('income = " >50K"').count()
        
        c2 = df.select('income').where('income =  " <=50K"').count()
        
        root.prediccion = ">50K" if c1>c2 else "<=50K"
        
        return
    
    categoria_entropy = {}
    
    for categoria in categorias:
        categoria_entropy[categoria] = get_feature_entropy(df, categoria)
    
    categoria_minima = min(categoria_entropy, key=categoria_entropy.get)
    
    if root.columna_hijos is None: ## Solamente entra en el root
        root.columna_hijos = categoria_minima
    
    valores_categoria = [valores[categoria_minima] for valores in df.select(categoria_minima).distinct().collect()]
    
    for valor in valores_categoria:
        
        root.hijos[valor] = Nodo(valor=valor, padre=root, columna_hijos=categoria_minima)
        
        df_ = df.where(f'{categoria_minima} = "{valor}"').drop(categoria_minima)
        
        categorias_aux = categorias.copy()
        categorias_aux.remove(categoria_minima)
        
        make_split(df_, root.hijos[valor], categorias_aux)

In [206]:
class Nodo:
    def __init__(self, valor, padre, prediccion=None, hijos=None, columna_hijos = None):
        self.valor = valor
        
        if padre == 'root':
            self.padre = None
        else:
            self.padre = padre
        
        self.hijos = {} if not hijos else hijos
        
        self.columna_hijos = columna_hijos
        
        self.prediccion = prediccion
        

In [207]:
def train(train_dataframe, columns=["sex", "occupation", "education", "workclass", "ageBucket"]):
    '''
    @return devuelve una estructura de datos que representa el árbol de decision
    '''
        
    arbol = Nodo(None, 'root')
    make_split(df=train_dataframe, root=arbol, categorias=columns)
    
    return arbol

# Version dinamica

En esta parte se intento hacer que una funcion pueda tomar una cantidad variable de columnas pero no se pudo resolver como pasarle un diccionario a un udf.
Se buscaron metodos y aparecieron algunas soluciones pero ninguna que se entiendiera o se pudiera utilizar.
Por lo tanto se expone el codigo nada mas.

In [220]:
arbol = train(df_adult_bucketized, columns=df_adult_bucketized.columns)

{'valor': None, 'padre': None, 'hijos': {}, 'columna_hijos': None, 'prediccion': None}
{'valor': ' Farming-fishing', 'padre': <__main__.Nodo object at 0x0000024596179EE0>, 'hijos': {}, 'columna_hijos': 'occupation', 'prediccion': None}
{'valor': ' State-gov', 'padre': <__main__.Nodo object at 0x0000024596482DF0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Federal-gov', 'padre': <__main__.Nodo object at 0x0000024596482DF0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Self-emp-not-inc', 'padre': <__main__.Nodo object at 0x0000024596482DF0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Local-gov', 'padre': <__main__.Nodo object at 0x0000024596482DF0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Private', 'padre': <__main__.Nodo object at 0x0000024596482DF0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Self-emp-inc', 'padre': <__main__.N

{'valor': ' State-gov', 'padre': <__main__.Nodo object at 0x00000245964675B0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Federal-gov', 'padre': <__main__.Nodo object at 0x00000245964675B0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Self-emp-not-inc', 'padre': <__main__.Nodo object at 0x00000245964675B0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Local-gov', 'padre': <__main__.Nodo object at 0x00000245964675B0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Private', 'padre': <__main__.Nodo object at 0x00000245964675B0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Self-emp-inc', 'padre': <__main__.Nodo object at 0x00000245964675B0>, 'hijos': {}, 'columna_hijos': 'workclass', 'prediccion': None}
{'valor': ' Transport-moving', 'padre': <__main__.Nodo object at 0x0000024596179EE0>, 'hijos': {}, 'columna_hijos': 'occupation', 'predicc

In [41]:
def imprimir_arbol(arbol, tab):
    
    if arbol.prediccion is None:
        print('|', tab*'►', arbol.valor)
        for _, hijo in arbol.hijos.items():
            imprimir_arbol(hijo, tab+1)
    else:
        print('|', tab*'--', arbol.valor, '►', arbol.prediccion)
    

In [ ]:
# imprime el arbol resultante
imprimir_arbol(arbol, 0)

In [160]:
#Version dinamica (no fucionante)
def predict(tree, train_dataframe, columns=["sex", "occupation", "education", "workclass", "ageBucket"]):
    '''
    @param tree la estructura de datos que representa el árbol de decisión.
    @ return un dataframe con todos los datos de train_dataframe con una columna adicional que representa la probabilidad de que el income sea >50K. 
    '''
    
    def give_prediction(columns):
        
        kwargs={}
        for col_name, col in zip(columns_name, columns):
            kwargs[col_name] = col
#         for key, value in kwargs:
#             kwargs[key] = kwargs[key][key]
        
        #p.append(str(kwargs))
        
        nodo_actual = tree.copy()
        
        while(nodo_actual.prediccion is None):
            
            if nodo_actual.hijos.get(kwarg.get(nodo_actual.columna_hijos)) is not None:
                nodo_actual = nodo_actual.hijos.get(kwarg.get(nodo_actual.columna_hijos))
            else:
                return "";
        
        return nodo_actual.prediccion
        
    
    
    predict_udf = F.udf(give_prediction, StringType())

    return train_dataframe.withColumn("prediction", predict_udf(F.lit([F.col(column) for column in columns])))

# Version hardcoded

Dado que no se pudo hacer la version dinamica, se intento hacer una version hardcodeada. Debajo se puede ver la version que tiene las 5 columnas que estabamos utilizando, pero como esta version demora muchas horas para hacer un simple testeo se tuvo que hacer una con menos columnas.
De todas formas por algun error que no se pudo arreglar, al hacer el .show() no aparece el resultado esperado.

In [189]:
#Version hardcoded (de 5 columnas)
import copy
def predict(tree, train_dataframe):
    '''
    @param tree la estructura de datos que representa el árbol de decisión.
    @ return un dataframe con todos los datos de train_dataframe con una columna adicional que representa la probabilidad de que el income sea >50K. 
    '''
    
    def give_prediction(sex, occupation, education, workclass, ageBucket):
                
        kwarg = {"sex":sex, "occupation":occupation,  "education": education, "workclass": workclass, "ageBucket": ageBucket}    
        
        nodo_actual = copy.deepcopy(tree)
        
        while(nodo_actual.prediccion is None):
            
            if nodo_actual.hijos.get(kwarg.get(nodo_actual.columna_hijos)) is not None:
                nodo_actual = nodo_actual.hijos.get(kwarg.get(nodo_actual.columna_hijos))
            else:
                return "";
        
        return nodo_actual.prediccion
        
    
    
    predict_udf = F.udf(give_prediction, StringType())

    return train_dataframe.withColumn("prediction", predict_udf(F.col("sex"), F.col("occupation"),F.col("education"), F.col("workclass"),F.col("ageBucket")))

In [1]:
# Version hardcoded (2 columnas)
import copy
def predict(tree, train_dataframe):
    '''
    @param tree la estructura de datos que representa el árbol de decisión.
    @ return un dataframe con todos los datos de train_dataframe con una columna adicional que representa la probabilidad de que el income sea >50K. 
    '''
    
    def give_prediction(occupation, workclass):
        kwarg = {"occupation":occupation,"workclass": workclass}    
        nodo_actual = copy.deepcopy(tree)
        
        while(nodo_actual.prediccion is None):
            
            if nodo_actual.hijos.get(kwarg.get(nodo_actual.columna_hijos)) is not None:
                nodo_actual = nodo_actual.hijos.get(kwarg.get(nodo_actual.columna_hijos))
            else:
                return ""
        
        return nodo_actual.prediccion
    
    predict_udf = F.udf(give_prediction, StringType())

    return train_dataframe.withColumn("prediction", predict_udf(F.col("occupation"), F.col("workclass")))

In [285]:
df_prediction = predict(arbol, df_adult_bucketized)

In [ ]:
df_prediction.show()

<i>[Extra credit y alumnos masters]</i>
    
5.  Mejorar la implementación de su algoritmo evitando hacer el split cuando no se logra un mínimo de Information Gain ( https://en.wikipedia.org/wiki/Information_gain_in_decision_trees   )


In [ ]:
#